In [4]:
import requests
from bs4 import BeautifulSoup
import os
import json
# import sys
# from pathlib import Path
# sys.path.append(str(Path.cwd().parent))
# from utils.scrapper import fetch_website_contents, fetch_website_links
from dotenv import load_dotenv
from openai import OpenAI
runLocalModels=False
from IPython.display import Markdown, display, update_display

ModuleNotFoundError: No module named 'openai'

In [38]:
def get_links_from_website(url):
    response = requests.get(url)
    content = BeautifulSoup(response.content,"html.parser")
    links= [link.get("href") for link in content.find_all("a")]
    valid_links = list(filter(lambda x: x.startswith("http"), links))
    return valid_links

In [39]:
load_dotenv(override=True)
api_key=os.getenv("OPENAI_API_KEY")

if api_key and api_key.startswith("sk-proj") and len(api_key)>10 :
    print("api key exists")
else:
    print("no api key")


api key exists


In [40]:
link_system_prompt = """you are provided with list of urls found in a web page
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}"""

In [41]:
def get_user_prompt(url):
    user_prompt = f"""Here is the list of urls found on the website (blog) {url}
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.

    Links (some might be relative links):
    """
    links=get_links_from_website(url)
    user_prompt+="\n".join(links)
    return user_prompt


In [42]:
def get_ai_instance():
    if requests.get("http://localhost:11434").ok and runLocalModels:
        MODEL ="gpt-oss:20b"
        OLLAMA_BASE_URL = "http://localhost:11434/v1"
        ai_instance = OpenAI(base_url=OLLAMA_BASE_URL,api_key="ollama")
    else:
        MODEL="gpt-5-nano"
        ai_instance = OpenAI()
    return (ai_instance,MODEL)

In [47]:
def call_ai_to_get_relavent_links(url,ai,model):
    print(f"Selecting relevant links for {url} by calling {model} on local mode {runLocalModels}")
    response = ai.chat.completions.create(
    model=model,
    messages=[{"role" : "system","content": link_system_prompt},
             {"role" : "user" , "content" : get_user_prompt(url)}],
    response_format={"type" : "json_object"})
    result= response.choices[0].message.content
    links = json.loads(result)
    print(f"found {len(links['links'])} relevant links")
    return links

In [48]:
links = call_ai_to_get_relavent_links("https://huggingface.co/",get_ai_instance()[0],get_ai_instance()[1])

Selecting relevant links for https://huggingface.co/ by calling gpt-5-nano on local mode False
found 5 relevant links


In [33]:
content = scrapper.fetch_website_contents("https://huggingface.co/")

In [49]:
def fetch_website_and_contents(url):
    contents = scrapper.fetch_website_contents(url)
    relevant_links = links
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n###{link['type']}\n"
        result += scrapper.fetch_website_contents(link['url'])
    return result

In [50]:
print(fetch_website_and_contents("https://huggingface.co/"))

## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
microsoft/VibeVoice-Realtime-0.5B
Updated
1 day ago
•
67.7k
•
626
Tongyi-MAI/Z-Image-Turbo
Updated
2 days ago
•
233k
•
2.46k
zai-org/GLM-4.6V-Flash
Updated
about 18 hours ago
•
10k
•
276
zai-org/GLM-4.6V
Updated
1 day ago
•
1.31k
•
233
deepseek-ai/DeepSeek-V3.2
Updated
9 days ago
•
40.7k
•
852
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.29k
Z Image Turbo
🏃
1.29k
Generate images from text prompts
Running
on
Zero
458
Z Image Turbo
🖼
458
Generate stunning images from text prompts
Running
170
Evaluation Guidebook
📝
170
Display evaluation metrics for LLM benchmarks
Running
on
Zero
MCP
Featured
883
Dream-wan2-2-faster-Pro
🎬

In [54]:
broucher_gen_sys_prompt="""You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information."""

In [51]:
def get_broucher_user_prompt(companyname, url):
    user_prompt = f"""you are looking at a company called: {companyname}, here are the contents of its landing page and relevant pages; 
    use this information to generate a short broucher, give in a markdown without code blocks"""
    user_prompt += fetch_website_and_contents(url)
    user_prompt = user_prompt[:5_000]
    return user_prompt

In [60]:
def generate_company_broucher(companyname, url, ai, model):
    response = ai.chat.completions.create(
        model= model,
        messages = [{"role":"system", "content" : broucher_gen_sys_prompt},
               {"role" : "user", "content" : get_broucher_user_prompt(companyname,url)}])
    result = response.choices[0].message.content
    display(Markdown(result))
    

In [61]:
generate_company_broucher("HuggingFace", "https://huggingface.co/",get_ai_instance()[0],get_ai_instance()[1])

Hugging Face — The AI community building the future

The platform where the machine learning community collaborates on models, datasets, and applications.

Key value proposition
- The Home of Machine Learning: create, discover, and collaborate on ML faster.
- The collaboration platform: host and collaborate on unlimited public models, datasets, and applications.
- Move faster with the HF Open source stack.
- Explore all modalities: text, image, video, audio, or even 3D.
- Build your portfolio: share your work with the world and grow your ML profile.
- Sign Up to join the community, publish your work, and accelerate your ML journey.

What we offer
- A huge ecosystem: browse 1M+ models, plus Spaces and datasets.
- Spaces: running applications and demos (examples include image generation from prompts, evaluation guides, video generation from images, and more).
- Fully managed deployment: Inference Endpoints to productionize models with minimal setup.
- One-click deployment: import models from Hugging Face or browse a hand-picked catalog.
- Compatibility with leading AI stacks: vLLM, TGI, SGLang, TEI, and custom containers.
- Seamless Hugging Face Hub integration: fast, secure model weight downloads.
- Future-proof AI stack: stay current with the latest frameworks and optimizations without messy upgrades.
- Pricing options: Self-Serve, pay-as-you-go, billed per minute for Inf­erence Endpoints.

Inference Endpoints — produce at scale
- Fully managed infrastructure: no Kubernetes or CUDA setup required.
- Autoscaling: scales with traffic to optimize costs.
- Observability: comprehensive logs and metrics to understand and debug models.
- Flexible inference engines: choose from vLLM, TGI, SGLang, TEI, or custom containers.
- Significantly easier production deployment: one-click imports from the catalog, fast deployment, and secure weights transfer.
- Per-minute pricing: a transparent, usage-based model.

Pricing
- Self-Serve, pay-as-you-go model for Inference Endpoints.
- Pay for what you use, per minute.

Community, customers, and partners
- Trusted by teams running AI models on Inference Endpoints (leading organizations deploy with our platform).
- A thriving community of researchers, developers, and data scientists building together.
- Enterprise and team offerings to equip organizations with the most advanced platform for AI deployment.

Careers and culture
- The AI community building the future — that mission underpins who we are and how we work.
- Founded in 2016; privately held with a growing team (collaborative culture, openness, and a focus on democratizing artificial intelligence).
- Current openings available on our careers page; join a fast-growing team focused on machine learning, NLP, and deep learning.
- LinkedIn presence highlights our size (hundreds of employees) and focus areas.

Get involved
- Sign up or log in to join the community, publish work, and access tools.
- Browse the catalog of models, datasets, and applications.
- Learn more about Inference Endpoints and Enterprise options.
- Explore enterprise solutions designed for teams and organizations.

Website navigation highlights
- Models, Datasets, Spaces, Community, Docs, Enterprise, Pricing
- Explore AI Apps or Browse 1M+ models
- Inference Endpoints, Catalog, and a growing catalog of ready-to-deploy models
- Pricing and plans for individual users to large teams

If you’d like more details or a tailored overview for prospective investors or potential hires, I can tailor this brochure to emphasize specific aspects (technical capabilities, customer stories, or career paths) and include direct calls to action.

In [69]:
def stream_brochure(company_name, url,ai, model):
    stream = ai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": broucher_gen_sys_prompt},
            {"role": "user", "content": get_broucher_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [70]:
stream_brochure("HuggingFace", "https://huggingface.co/",get_ai_instance()[0],"gpt-4.1-mini")

# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a vibrant AI community and technology company dedicated to building the future of machine learning. Founded in 2016, Hugging Face specializes in advancing machine learning, natural language processing (NLP), and deep learning through an open and collaborative platform. With a team of 51-200 passionate employees, the company is privately held and actively shaping the democratization of artificial intelligence.

---

## What We Offer

### The Home of Machine Learning Collaboration  
Hugging Face provides the ultimate platform where the global machine learning community can create, discover, and collaborate on a vast array of models, datasets, and applications. With over 1 million models and 250,000 datasets accessible, users accelerate ML research and development across multiple modalities including text, image, video, audio, and even 3D.

### Key Features:
- **Public Hosting & Collaboration:** Host unlimited public models, datasets, and applications.
- **Wide Variety of Modalities:** Support for text generation, image generation, video synthesis, and more.
- **Machine Learning Applications:** Over 400,000 AI apps available to explore and build upon.
- **Open Source Stack:** Move faster with Hugging Face’s open-source tools and libraries.
- **Portfolio Building:** Share your work globally and build your professional ML profile.

### Enterprise & Compute Solutions  
For organizations and teams seeking to scale AI deployment, Hugging Face offers paid compute resources and enterprise-grade solutions. Their **Inference Endpoints** service allows seamless, one-click deployment of AI models on a fully managed platform with autoscaling, observability, and robust infrastructure—eliminating the hassles of backend configuration.

---

## Customers and Community

Hugging Face is trusted by industry-leading teams who deploy AI models on their managed inference platform. The community includes researchers, developers, startups, and enterprises, all collaborating through a shared platform to push the boundaries of AI innovation.

---

## Company Culture

Hugging Face fosters a culture of openness, collaboration, and continuous learning. They emphasize:
- Building **inclusive AI communities** where knowledge is freely shared.
- Encouraging **creativity and experimentation** in machine learning projects.
- Supporting contributors and users at all skill levels to build meaningful AI applications.
- Prioritizing transparent and **open source development** to democratize AI.

---

## Careers at Hugging Face

Are you passionate about AI and eager to work in a fast-growing, innovative environment? Hugging Face offers exciting career opportunities in software development, research, and AI engineering. Join a dedicated team that is shaping the future of machine learning and contributing actively to global AI advancement.

Explore current job openings and apply to become part of the AI community building the future.

---

## Connect With Us

- Website: [huggingface.co](https://huggingface.co)  
- LinkedIn: [Hugging Face on LinkedIn](https://www.linkedin.com/company/huggingface)  
- Join the community, collaborate on models, and start building your AI applications today!

---

**Hugging Face** – Empowering the AI community to build the future of machine learning.